In [94]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pyexcel_ods import get_data

# read data
data_giro = get_data("data/1032515684.csv.ods", encoding = 'ISO-8859-1', sep = ';')
df_giro = pd.DataFrame(data_giro['Sheet1'])

# split data
df_giro = pd.concat([df_giro[0].str.split(';', expand=True)], axis=1)
# move header from first row
df_giro.columns = df_giro.iloc[0, :]
df_giro = df_giro.iloc[1:]

# inspect table
print(df_giro.columns)
print(df_giro.shape)

# drop columns not needed for analysis
df_giro = df_giro.iloc[:, [0, 3, 4, 5, 7]]

# rename columns
df_giro.columns = ["date", 'debitor', "subject", "account", "amount"]
# merge debitor and subject column for more information
df_giro['text'] = df_giro['debitor'] + df_giro['subject']

# get NA columns
na_index  = (df_giro["amount"].isna().index[df_giro["amount"].isna() == False].tolist())

# concatenate subjects which actually belong together
s = " "
for i in range(0, len(na_index) - 1):
    df_giro.iloc[na_index[i], 1] = s.join(df_giro.iloc[range(na_index[i], na_index[i+1]), 1])

# remove rows with NA
df_giro = df_giro[pd.notnull(df_giro['amount'])]

# replace NULL with 0
df_giro = df_giro.fillna('0,0')

# convert to numerics
for i in range(0, df_giro.shape[0]):
    df_giro.iloc[i, 4] = df_giro.iloc[i, 4].replace('.', '')
    df_giro.iloc[i, 4] = df_giro.iloc[i, 4].replace('"', '')
    df_giro.iloc[i, 4] = float(df_giro.iloc[i, 4].replace(',', '.'))
    
# drop incomes
df_giro = df_giro[df_giro['amount'] < 0]

# drop credit card settlements
df_giro = df_giro[df_giro['debitor'] != '"KREDITKARTENABRECHNUNG"']

# only relevant columns for merging
df_giro = df_giro.iloc[:, [0, 5, 4]]

Index(['Buchungstag', '"Wertstellung"', '"Buchungstext"',
       '"Auftraggeber / Begünstigter"', '"Verwendungszweck"', '"Kontonummer"',
       '"BLZ"', '"Betrag (EUR)"', '"Gläubiger-ID"', '"Mandatsreferenz"',
       '"Kundenreferenz"', ''],
      dtype='object', name=0)
(320, 12)


In [95]:
# read data
data_credit = get_data("data/4748.csv.ods", encoding = 'ISO-8859-1', sep = ';')
df_credit = pd.DataFrame(data_credit['Sheet1'])

# split data
df_credit = pd.concat([df_credit[0].str.split(';', expand=True)], axis=1)

# move header from first row
df_credit.columns = df_credit.iloc[0, :]
df_credit = df_credit.iloc[1:]

# inspect table
print(df_credit.columns)
print(df_credit.shape)

# drop columns not needed for analysis
df_credit = df_credit.iloc[:, [2, 3, 4]]

# rename columns
df_credit.columns = ["date", 'text', "amount"]

# convert to numerics
for i in range(0, df_credit.shape[0]):
    df_credit.iloc[i, 2] = df_credit.iloc[i, 2].replace('.', '')
    df_credit.iloc[i, 2] = df_credit.iloc[i, 2].replace('"', '')
    df_credit.iloc[i, 2] = float(df_credit.iloc[i, 2].replace(',', '.'))
    
# drop credit card settlements / incomes
#df_credit = df_credit[df_credit['text'] != 'Ausgleich']
df_credit = df_credit[df_credit['amount'] < 0]

Index(['Umsatz abgerechnet und nicht im Saldo enthalten', '"Wertstellung"',
       '"Belegdatum"', '"Beschreibung"', '"Betrag (EUR)"',
       '"Ursprünglicher Betrag"', ''],
      dtype='object', name=0)
(222, 7)


In [99]:
# merge data frames
df = df_giro.append(df_credit) 

# cluster dictionary
cluster = {
    'grocery': ['ALDI', 'APOTHEKE', 'DM', 'EDEKA', 'FOODORA', 'HELLOFRESH', 'KAUFLAND', 'Kaufland',
                'Lidl', 'LIDL', 'Lieferando', 'NAHKAUF', 'ORTERER', 'REAL', 'REWE'],
    
    'travel':     [
                    # tankstellen
                    'AGIP', 'ARAL', 'BP', 'Eni', 'OMV', 'Rastanlage', 'SHELL', 'TANK', 'Tankstelle',
                    # berge
                    'Gletscherbahn', 'Karwendelbahn', 'MIEDERSMIEDERS', 'RATSCHINGS',
                    # verkehrsmittel
                    'AUTOMATENMUENCHEN', 'DB', 'MAUTSTELLE', 'Scheffau', 
                    # slowenien
                    'HRUSICAHRUSICA', 'KITCHEBLED', 'LJUBLJANA',
                    # spanien
                    'CORDOBA', 'GRANADA', 'KARTMONIGA', 'MADRID', 'SEVILLA', 'TRAINLINE', 
                    # amsterdam
                    'AMSTERDAM', 'Amsterdam', 'MYLAPS',
                    # unterkunft
                    'AIRBNB', 'HMS HUETTEN-MIET', 'HOLZKNECHTHUETTE', 'HOTEL', 'Hotel',
                    # rest
                    'CHECKOUTMUENCHEN', 'Coop', 'FEDERICODESENZANO', 'KILLY', 'LAIMBBOLZANO', 
                    'PP.9048.PP', 'SACATUENTRADA', 'WALLISELLEN', 'Zurich', 'ZVKDZ'
                  ], 
    
    'cash':    ['COMMERZBANK', 'HVB', 'hypo', 'KSK', 'MUENCHNER BANK', 'POSTBANK', 'RAIFFEISENBANK', 
                'SPARDA', 'Sparkasse', 'STADTSPARKASSE', 'VR-BANK'],
    
    'debt': ['Bundeskasse'],
    
    'run_costs': [
                  # miete/wohnen
                  'GWG ST.WOHN', 'M-net', 'RENKER',
                  # versicherungen
                  'ALLIANZ AG',  'BAYER. LANDESBRANDVERS.', 'HE-1008-2831', 'UNION', 'VHV Allgemeine Versicherung',
                  # job
                  'ALLIANZ DEUTSCHLAND', 'BAYERISCHE LANDESBRANDVERS', 'VERKEHRSGESELLS',
                  # rest
                  'E-Plus', 'MTV Muenchen', 'SpotifyStockholm'
                  ],
    
    'shopping': ['AMAZON', 'adidas', 'BAUHAUS', 'DALLMAYR', 'ELBENWALD', 'HUGENDUBEL', 'HULA', 'Jack + Jones', 
                 'KARSTADT', 'LUDWIG BECK', 'MINIT', 'NINTENDO', 'OUTFITTERY', 'SCHUSTER', 'STRATIC'],
    
    'property': ['Abfallwirtschaftsverband', 'KAMINKEHRERMEISTER', 'Schmutzwassergeb.', 'Simbach'],
    
    'goout': ['Banyan', 'BRAEUSTUBEN', 'CHI THU', 'DONALDS', 'HANOI', 'KANSHA', 'Nam Giao', 'NAMASTE',
              'Ngoc', 'Paulaner', 'PEPENERO', 'RESTAURANT', 'Restaurant', 'SEESTUEBERL', 'Shadow Bean', 
              'SUBWAY', 'TAKUMI', 'UBER']
}

df['cluster'] = ''

# assign bookings to cluster
for x in df['text']:
    for y in cluster:
        for z in cluster[y]:
            if (z in x):
                df.loc[df['text'] == x, 'cluster'] = y
                
# show items not assigned
if len(df.loc[df['cluster'] == '', 'text']) == 0:
    print('all items assigned')
else:
    print(df.loc[df['cluster'] == '', 'text'])
    
df.loc[df['cluster'] == '', 'cluster'] = 'other'

46     "ROBERT HAGER""MS Geld               DATUM 16....
56     "FINANZKASSE MUENCHEN""Einkommenssteuer 2018 -...
192    "POLIZEIVERWALTUNGSAMT""DATUM 28.05.2019,     ...
214    "ROBERT HAGER""ETF Geld              DATUM 29....
221    "SCHLOSSWALD GMBH""Erinnerungstafel Auftrag-Nr...
232    "STADT NITTENAU""Bestattung Engelbert Hager - ...
240    "SPORTPARK//Muenchen/DE / SPORTPARK""2019-04-0...
258    "KREISVERWALTUNGSREF. MCHN.//Muenchen/DE / KRE...
307    "JANA BETZOLD""Kochkurs  ORIENTAL SOULFOOD  - ...
33                  "EB *HERBSTTAGUNG DES G801-413-7200"
Name: text, dtype: object


In [100]:
df.loc[df['cluster'] == 'other', ].to_csv('other.csv')
df.loc[df['cluster'] == 'run_costs', ].to_csv('run_costs.csv')

In [97]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = df['cluster'], 
    y = df['amount'], 
    hovertext = df['text'], 
    hoverinfo = "text", 
    marker_color = 'rgb(158, 202, 225)', 
    marker_line_color = 'rgb(8, 48, 107)',
    marker_line_width = 1.5, 
    opacity = 0.6
))

fig.update_layout(title_text = 'Expenses')


fig.show()